In [1]:
from rdflib import Graph

In [2]:
g = Graph()
g = g.parse("data/music_graph.ttl")

In [3]:
qres = g.query("""
    SELECT ?s ?p ?o
    WHERE {
        ?s ?p ?o
    }
""")

f"Graph size: {len(qres)}"

'Graph size: 656350'

1. Какой самый популярный жанр музыки среди выпущенных треков?

In [4]:
qres = g.query("""
    SELECT ?genreName (COUNT(?track) AS ?count)
    WHERE {
        ?track rdf:type ns1:Track .
        ?track ns1:hasGenre ?genre .
        ?genre ns1:name ?genreName .
    }
    GROUP BY ?genreName
    ORDER BY DESC(?count)
    LIMIT 5
""")

for i, row in enumerate(qres):
    genre, tracks_count = row
    print(f"{genre}, {tracks_count}")

Rock, 12911
Electronic, 4769
Metal, 3914
Pop, 1386
Rap, 1047


2. Какой средний показатель танцевальности у топ-100 треков по популярности?

In [5]:
qres = g.query("""
    SELECT (AVG(?danceability) AS ?averageDanceability)
    WHERE {
        SELECT ?danceability
        WHERE {
            ?track rdf:type ns1:Track .
            ?track ns1:popularity ?popularity .
            ?track ns1:danceability ?danceability .
        }
        ORDER BY DESC(?popularity)
        LIMIT 100
    }
""")

qres2 = g.query("""
    SELECT (AVG(?danceability) AS ?averageDanceability) 
    WHERE {
        ?track rdf:type ns1:Track .
        ?track ns1:danceability ?danceability .
    }
""")

for row1, row2 in zip(qres, qres2):
    avg_danceability_100 = row1[0].value
    avg_danceability_all = row2[0].value
    print(f"{avg_danceability_100}, {avg_danceability_all}")

0.6021900000000003, 0.48235880655144864


3. Альбом какой группы содержит наибольшее количество песен?

In [6]:
qres = g.query("""
    SELECT ?performerName ?albumName ?tracks
    WHERE {
        ?album rdf:type ns1:Album .
        ?album ns1:albumType "album" .
        ?album ns1:tracksTotal ?tracks .
        ?album ns1:performedBy ?performer .
        ?album ns1:name ?albumName .
        ?performer ns1:name ?performerName
    }
    ORDER BY DESC(?tracks) LIMIT 1
""")

for i, row in enumerate(qres):
    performer_name, album_name, tracks_count = row
    print(f"{performer_name}, {album_name}, {tracks_count}")

Serge Gainsbourg, Intégrale, 271


4. Какая группа выпустила больше всего альбомов за 2006 год?

In [7]:
qres = g.query("""
    SELECT ?performerName (COUNT(?album) as ?albumsInYear)
    WHERE {
        ?album rdf:type ns1:Album .
        ?album ns1:albumType "album" .
        ?album ns1:releaseYear "2006" .
        ?album ns1:performedBy ?performer .
        ?performer ns1:name ?performerName
    }
    GROUP BY ?performerName
    ORDER BY DESC(?allPerformerTracks) LIMIT 1
""")

for i, row in enumerate(qres):
    performer_name, tracks_count = row
    print(f"{performer_name}, {tracks_count}")

Patrick Watson, 4


4. Какая группа выпустила больше всего песен за 2014 год?

In [8]:
qres = g.query("""
    SELECT ?performerName (SUM(?tracks) as ?allPerformerTracks)
    WHERE {
        ?album rdf:type ns1:Album .
        ?album ns1:albumType "album" .
        ?album ns1:tracksTotal ?tracks .
        ?album ns1:releaseYear "2014" .
        ?album ns1:performedBy ?performer .
        ?performer ns1:name ?performerName
    }
    GROUP BY ?performerName
    ORDER BY DESC(?allPerformerTracks) LIMIT 1
""")

for i, row in enumerate(qres):
    performer_name, tracks_count = row
    print(f"{performer_name}, {tracks_count}")

Franz Schubert, 43
